# Purpose

Take an input STL file and sort its faces based on [TriMesh](https://github.com/mikedh/trimesh) facets into inlet, outlet, wall, and top and bottom faces. Write each type of sorted faces to individual STL files. Also collect the faces orthogonal to the x-y plane and write to a combined STL file used by `cfmesh` to create a 1-cell thick mesh suitable for 2D OpenFoam simulations.

# Imports

In [1]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import trimesh

# Code

In [2]:
def correct_stl_string(original_str, name):
    """Make corrections to STL string returned by 
    'trimesh.exchange.stl.export_stl_ascii()' so that there
    is a defined name and a final blank line. These are needed to
    create a valid STL file for use in OpenFoam.
    """
    temp = original_str.split("\n")
    temp[0] += f"{name}"
    temp[-1] += f" {name}\n"
    return "\n".join(temp)

    
def write_stl_string_to_ascii_stl_file(stl_string, filename, write_directory=None):
    if write_directory is None:
        output_file = Path(filename)
    else:
        assert isinstance(write_directory, Path)
        output_file = write_directory / filename
    with open(output_file, "w") as f:
        f.write(stl_string)

def write_to_ascii_stl_file(mesh, write_directory=None):
    """Write 'trimesh.Trimesh' object to ascii STL file.
    """
    assert hasattr(mesh, "name")
    stl_string = correct_stl_string(
        trimesh.exchange.stl.export_stl_ascii(mesh), 
        mesh.name
    )
    write_stl_string_to_ascii_stl_file(stl_string, mesh.name + ".stl", write_directory)

def write_ascii_stl_file_for_side_faces(
    mesh, 
    sorted_facets, 
    included_submeshes = ["inlet", "outlet", "walls"], 
    write_directory=None
):
    """Write specified faces to ascii STL file.
    """
    submeshes = create_submeshes(mesh, sorted_facets)
    
    stl_strings = []
    for name, submesh in submeshes.items():
        if name not in included_submeshes:
            continue
        stl_strings.append(
            correct_stl_string(
                trimesh.exchange.stl.export_stl_ascii(submesh), 
                submesh.name
            )
        )
        
    stl_string = "".join(stl_strings)
    
    write_stl_string_to_ascii_stl_file(stl_string, "combined.stl", write_directory)

In [3]:
def calculate_facets_reverse_index(mesh):
    """Add new attribute to trimesh mesh,
    'facets_reverse_index_from_face_id', which is a
    reverse index dict such that given the id of a face,
    the corresponding facet to which that face belongs
    is returned. The facet is comprised of a numpy 
    array of face id's.
    """
    mesh.facets_reverse_index_from_face_id = {}

    for facet_id, facet in enumerate(mesh.facets):
        for face_id in facet:
            mesh.facets_reverse_index_from_face_id[face_id] = facet_id


In [4]:
def facet_vertices(mesh, facet_id):
    """Return a list of all vertices in a facet.
    Since a facet is comprised of faces which in turn
    are each triangles, the number of vertices is 3 * 
    (number of faces in facet).
    """
    vertices = []
    for face_id in mesh.facets[facet_id]:
        for vertex in mesh.vertices[mesh.faces[face_id]]:
            vertices.append([vertex[0], vertex[1], vertex[2]])    
    return vertices

In [5]:
def face_vertices(mesh, face_id):
    """Return a list of all vertices in a face.
    Since a face is a triangle, the list always contains
    3 vertices.
    """
    vertices = []
    for vertex in mesh.vertices[mesh.faces[face_id]]:
        vertices.append([vertex[0], vertex[1], vertex[2]])    
    return vertices


def create_submeshes(mesh, sorted_facets):
    """Create a 'trimesh.Trimesh' for each sorted facet in 'sorted_facets'.
    """
    submeshes = dict()
    for name, included_facets in sorted_facets.items():
    
        new_vertices = []
        new_faces = []

        vertex_index = 0
        for j, facet_id in enumerate(included_facets):
            
            for i, face_id in enumerate(mesh.facets[facet_id]):
                temp_vertices = face_vertices(mesh, face_id)
                for vertex in temp_vertices:
                    new_vertices.append(vertex)
                new_faces.append([temp for temp in range(vertex_index, vertex_index + 3)])
                vertex_index += 3

        submeshes[name] = trimesh.Trimesh(vertices=new_vertices, faces=new_faces)
        submeshes[name].name = name
        
    return submeshes

def create_and_write_STL_boundary_files(mesh, sorted_facets):
    """Given a 'trimesh.Trimesh' and a dict of its sorted facets,
    create a new `trimesh.Trimesh` for each type of sorted facet
    and write to its own ascii STL file. The new meshes and files
    are named according to the keys in 'sorted_facets'.
    """
    submeshes = create_submeshes(mesh, sorted_facets)
    for _, submesh in submeshes.items():
        write_to_ascii_stl_file(submesh, write_directory=None)
    

# Unit vector definitions

In [6]:
norm_minus_x = np.array([-1, 0, 0], dtype=np.float32)
norm_plus_x = np.array([1, 0, 0], dtype=np.float32)
norm_minus_y = np.array([0, -1, 0], dtype=np.float32)
norm_plus_y = np.array([0, 1, 0], dtype=np.float32)
norm_minus_z = np.array([0, 0, -1], dtype=np.float32)
norm_plus_z = np.array([0, 0, 1], dtype=np.float32)

# Load STL

In [7]:
mesh = trimesh.load_mesh("straight_channel_0deg.stl")

In [8]:
print(f"{mesh.vertices.shape=}")
print(f"{mesh.faces.shape=}")
print()
print(f"{len(mesh.facets)=}")
print(f"{len(mesh.facets_normal)=}")

calculate_facets_reverse_index(mesh)
print(f"{len(mesh.facets_reverse_index_from_face_id)=}")


mesh.vertices.shape=(8, 3)
mesh.faces.shape=(12, 3)

len(mesh.facets)=6
len(mesh.facets_normal)=6
len(mesh.facets_reverse_index_from_face_id)=12


# Scale from mm to m

In [9]:
mesh.apply_scale(1e-3)

<trimesh.Trimesh(vertices.shape=(8, 3), faces.shape=(12, 3))>

## Save scaled original stl file

In [10]:
# Write scaled mesh to STL file
mesh.name = "original_scaled"
write_to_ascii_stl_file(mesh)

# Plan - sort faces

Loop over facets and collect facet id's.

- Normal = +z &rarr; top
- Normal = -z &rarr; bottom
- Normal = -x  &rarr; inlet
- Normal = +x  &rarr; outlet
- Otherwise &rarr; wall

# Sort triangles (vectors)

## Define points for sorting outlet facets

In [11]:
# mesh.bounding_box.bounds

In [12]:
# bb_min = mesh.bounding_box.bounds[0]
# bb_max = mesh.bounding_box.bounds[1]

# print(f"{bb_min=}")
# print(f"{bb_max=}")

# important_points = {
#     "inlet": [
#         bb_min[0], bb_max[1], 0.5 * (bb_min[2] + bb_max[2]),
#     ],
#     "outlet": [
#         bb_max[0], bb_min[1], 0.5 * (bb_min[2] + bb_max[2]),
#     ],
#     "wall1": [
#         bb_min[0], bb_min[1], 0.5 * (bb_min[2] + bb_max[2]),
#     ],
#     "wall2": [
#         bb_max[0], bb_max[1], 0.5 * (bb_min[2] + bb_max[2]),
#     ],
# }
# important_points

## Sort facets

In [13]:
sorted_facets = {
    'inlet': [],
    'outlet': [],
    'empty_patch_top': [],
    'empty_patch_bottom': [],
    'walls': [],
}

already_sorted_facets = set()

# # Loop over defined points to identify outlet facets
# for facet_name, point in important_points.items():
#     (_, _, face_id) = mesh.nearest.on_surface([point])
#     facet_id = mesh.facets_reverse_index_from_face_id[face_id[0]]
#     if facet_name.startswith("wall"):
#         facet_name = facet_name[:-1] + "s"
#     sorted_facets[facet_name].append(facet_id)
#     already_sorted_facets.add(facet_id)
    
# print(f"{already_sorted_facets=}")

# Sort remaining facets based on surface normals
for facet_id, (facet, facet_normal) in enumerate(zip(mesh.facets, mesh.facets_normal)):
    # print(facet_id, facet, facet_normal)
    
    if facet_id in already_sorted_facets:
        continue
    
    if np.allclose(facet_normal, norm_minus_z):
        sorted_facets['empty_patch_bottom'].append(facet_id)
        
    elif np.allclose(facet_normal, norm_plus_z):
        sorted_facets['empty_patch_top'].append(facet_id)
        
    elif np.allclose(facet_normal, norm_minus_x):
        sorted_facets['inlet'].append(facet_id)
        
    elif np.allclose(facet_normal, norm_plus_x):
        sorted_facets['outlet'].append(facet_id)

    else:
        sorted_facets['walls'].append(facet_id)
                
sorted_facets

{'inlet': [5],
 'outlet': [3],
 'empty_patch_top': [0],
 'empty_patch_bottom': [1],
 'walls': [2, 4]}

## Break out faces and write to STL files

In [14]:
# create_and_write_STL_boundary_files(mesh, sorted_facets)

write_ascii_stl_file_for_side_faces(
    mesh, 
    sorted_facets
)